### Capstone 2- Bengali Grapheme Classification Project

This workbook is the first attempt at experimenting with a CNN for this project. I've followed this [starter code](https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn) for ideas. 

This is also the first attempt at using MLFlow for tracking deep learning experiments.

This set of experiments do not use image augmentation

In [1]:
#**********************************************************
#standard and ml imports
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
from keras import backend as K
K.tensorflow_backend._get_available_gpus() # ensuring keras uses GPU (rocket emoji)

#**********************************************************
# using custom scripts for creating model and loading data
import sys
sys.path.insert(1, './model/')

# helper function to automate model building
from model_creator import model_create
# helper function to import data
from data_loader import data_loader
# helper function to do image threshold filtering and resizing
from image_processor import image_processor_func
# helper function to start and track mlflow experiments
from experiments import run_experiment_with_callbacks
# function to output final predictions on submission set
from tester import test_func
# function to do data augmentation for multiple output convnets - from this kernel 
# https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn
from batch_generator import MultiOutputDataGenerator

Using TensorFlow backend.


In [2]:
# defining function to reset graph and set seeds - from Aurelien Gueron's book HandsOn ML
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
reset_graph()

Before we begin our experiments, it is good practice to define an experiment matrix. We define the matrix with the following parameters :
- convolutional kernel size
- pooling size
- dropout rate (we assume both dropouts share the same rate)
- number of convolutional layers
- number of convolutional filters

The first experiment matrix contains a total of 108 experiments

In [4]:
# loading the pre-defined experiment csv
experiment_matrix = pd.read_csv('./experiment_matrix.csv',index_col='Experiment',)
experiment_matrix.head()

,kernel_size,pool_size,dropout,conv_layers,conv_filters
Experiment,,,,,
1,6,2,0.25,3,50
2,6,2,0.25,4,50
3,6,2,0.25,5,50
4,5,2,0.25,3,50
5,5,2,0.25,4,50


Next, we read the project csv files which define the labels

In [5]:
# read the project csv files
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

Now, we start building our neural networks, start training and track training results using the MLFlow API. For this project, we use the MLFlow Keras Autologger

In [6]:
# define batch size and number of epochs for training
BATCH_SIZE = 100
EPOCHS = 50

In [8]:
# starting our experiments
import mlflow.keras
modelerrCount=0
experrCount=0
for index, experiment in experiment_matrix.iterrows():
    if index in [13,35,44]:
        # resetting computational graph
        reset_graph()
        # specifying title
        title = 'exp_{0}_aug_yes_kernel_size_{1}_pool_size_{2}_dropout_rate_{3}_conv_layers_{4}_conv_filters_{5}'.format(
        index,int(experiment.loc['kernel_size']),int(experiment.loc['pool_size']),experiment.loc['dropout'],int(experiment.loc['conv_layers']),
        int(experiment.loc['conv_filters']))
        try:
            print('Creating Model')
            # create model with early stopping enabled
            model,callbacks_list = model_create(input_shape=(68,118,1),conv_kernel_size=int(experiment.loc['kernel_size']),
                                 pool_size=int(experiment.loc['pool_size']),
                                dropout_rate1=experiment.loc['dropout'],
                                 dropout_rate2=experiment.loc['dropout'],
                                n_conv_layers=int(experiment.loc['conv_layers']),
                                 conv_nfilters=int(experiment.loc['conv_filters']),early_stopping=True,learningrate_reduction=True)
        except:
            # logging errors if any
            errCount+=1
            if errCount==1:
                with open('modelfailurelog.txt','w') as f:
                    f.write('Experiment {} failed - Check Model Setup'.format(title))
            else:
                with open('modelfailurelog.txt','a') as f:
                    f.write('Experiment {} failed - Check Model Setup'.format(title))
        print('Model Created')
        print(model.summary())
        print('Running Experiment-{}'.format(index))
        print("-------------------------------------")
        try:
            # running actual experiment
            run_experiment_with_callbacks(title,model,callbacks_list,batch_size=BATCH_SIZE,epochs=EPOCHS,df_dict=df_dict,data_augmentation=True)
        except:
            # logging errors, if any
            experrCount+=1
            if experrCount==1:
                with open('experimentfailurelog.txt','w') as f:
                    f.write('Experiment {} failed - Check Exp Setup'.format(title))
            else:
                with open('experimentfailurelog.txt','a') as f:
                    f.write('Experiment {} failed - Check Exp Setup'.format(title))
print('All experiments finished')

Creating Model
Model Created
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer (InputLayer)        (None, 68, 118, 1)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 68, 118, 50)  1300        input_layer[0][0]                
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 34, 59, 50)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 34, 59, 50)   62550       max_pooling2d_1[0][0]            
_______________________________________________________________

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505


Epoch 1/50
451/451 [==============================] - 27s 59ms/step - loss: 7.5363 - output_root_loss: 4.8251 - output_vowel_loss: 1.5496 - output_consonant_loss: 1.1616 - output_root_accuracy: 0.0316 - output_vowel_accuracy: 0.4659 - output_consonant_accuracy: 0.6149 - val_loss: 5.7932 - val_output_root_loss: 4.2480 - val_output_vowel_loss: 0.7433 - val_output_consonant_loss: 0.7873 - val_output_root_accuracy: 0.0701 - val_output_vowel_accuracy: 0.7411 - val_output_consonant_accuracy: 0.7176
Epoch 2/50
451/451 [==============================] - 25s 56ms/step - loss: 5.7571 - output_root_loss: 4.1184 - output_vowel_loss: 0.8612 - output_consonant_loss: 0.7775 - output_root_accuracy: 0.0762 - output_vowel_accuracy: 0.6987 - output_consonant_accuracy: 0.7290 - val_loss: 4.5614 - val_output_root_loss: 3.4887 - val_output_vowel_loss: 0.5553 - val_output_consonant_loss: 0.5060 - val_output_root_accuracy: 0.1468 - val_output_vowel_accuracy: 0.8038 - val_output_consonant_accuracy: 0.8126
Epoc

451/451 [==============================] - 25s 57ms/step - loss: 2.0985 - output_root_loss: 1.3495 - output_vowel_loss: 0.4076 - output_consonant_loss: 0.3414 - output_root_accuracy: 0.6080 - output_vowel_accuracy: 0.8593 - output_consonant_accuracy: 0.8831 - val_loss: 1.2800 - val_output_root_loss: 0.8883 - val_output_vowel_loss: 0.2128 - val_output_consonant_loss: 0.1789 - val_output_root_accuracy: 0.7379 - val_output_vowel_accuracy: 0.9299 - val_output_consonant_accuracy: 0.9406
Epoch 18/50
451/451 [==============================] - 25s 56ms/step - loss: 2.0582 - output_root_loss: 1.3194 - output_vowel_loss: 0.4024 - output_consonant_loss: 0.3364 - output_root_accuracy: 0.6150 - output_vowel_accuracy: 0.8626 - output_consonant_accuracy: 0.8831 - val_loss: 1.3600 - val_output_root_loss: 0.9668 - val_output_vowel_loss: 0.2019 - val_output_consonant_loss: 0.1907 - val_output_root_accuracy: 0.7222 - val_output_vowel_accuracy: 0.9331 - val_output_consonant_accuracy: 0.9365
Epoch 19/50
45

451/451 [==============================] - 25s 56ms/step - loss: 1.7176 - output_root_loss: 1.0796 - output_vowel_loss: 0.3443 - output_consonant_loss: 0.2937 - output_root_accuracy: 0.6795 - output_vowel_accuracy: 0.8834 - output_consonant_accuracy: 0.8997 - val_loss: 1.1348 - val_output_root_loss: 0.7909 - val_output_vowel_loss: 0.1727 - val_output_consonant_loss: 0.1677 - val_output_root_accuracy: 0.7783 - val_output_vowel_accuracy: 0.9466 - val_output_consonant_accuracy: 0.9438
Epoch 34/50
451/451 [==============================] - 25s 56ms/step - loss: 1.6949 - output_root_loss: 1.0623 - output_vowel_loss: 0.3368 - output_consonant_loss: 0.2958 - output_root_accuracy: 0.6846 - output_vowel_accuracy: 0.8837 - output_consonant_accuracy: 0.8996 - val_loss: 1.1084 - val_output_root_loss: 0.7642 - val_output_vowel_loss: 0.1788 - val_output_consonant_loss: 0.1635 - val_output_root_accuracy: 0.7797 - val_output_vowel_accuracy: 0.9403 - val_output_consonant_accuracy: 0.9456
Epoch 35/50
45

...

/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505
/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/keras.py:442: UserWarning: Logging to MLflow failed: Changing param values is not allowed. Param with key='learning_rate' was already logged with value='0.001' for run ID='def9f0222a6647a6b8a15ea3f1a46c78'. Attempted logging new value '6.25e-05'.
  try_mlflow_log(mlflow.log_param, 'learning_rate', lr)


451/451 [==============================] - 35s 78ms/step - loss: 1.1127 - output_root_loss: 0.6880 - output_vowel_loss: 0.2147 - output_consonant_loss: 0.2100 - output_root_accuracy: 0.8028 - output_vowel_accuracy: 0.9316 - output_consonant_accuracy: 0.9299 - val_loss: 0.5858 - val_output_root_loss: 0.3759 - val_output_vowel_loss: 0.0979 - val_output_consonant_loss: 0.1098 - val_output_root_accuracy: 0.8921 - val_output_vowel_accuracy: 0.9711 - val_output_consonant_accuracy: 0.9657
Epoch 2/50
451/451 [==============================] - 35s 77ms/step - loss: 1.0926 - output_root_loss: 0.6712 - output_vowel_loss: 0.2145 - output_consonant_loss: 0.2069 - output_root_accuracy: 0.8041 - output_vowel_accuracy: 0.9323 - output_consonant_accuracy: 0.9320 - val_loss: 0.5830 - val_output_root_loss: 0.3727 - val_output_vowel_loss: 0.0991 - val_output_consonant_loss: 0.1090 - val_output_root_accuracy: 0.8911 - val_output_vowel_accuracy: 0.9693 - val_output_consonant_accuracy: 0.9661
Epoch 3/50
451/

Compressing Images
-------------------------------------
Transformation done
-------------------------------------
Augmenting Input Data
-------------------------------------
Training model on parquet file #4
-------------------------------------
Epoch 1/50


/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/utils/autologging_utils.py:60: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  all_param_names, _, _, all_default_values = inspect.getargspec(fn)  # pylint: disable=W1505
/home/jayanth/anaconda3/envs/tf1_env/lib/python3.6/site-packages/mlflow/keras.py:442: UserWarning: Logging to MLflow failed: Changing param values is not allowed. Param with key='learning_rate' was already logged with value='0.001' for run ID='def9f0222a6647a6b8a15ea3f1a46c78'. Attempted logging new value '1.5625e-05'.
  try_mlflow_log(mlflow.log_param, 'learning_rate', lr)


451/451 [==============================] - 35s 78ms/step - loss: 1.0564 - output_root_loss: 0.6483 - output_vowel_loss: 0.2115 - output_consonant_loss: 0.1966 - output_root_accuracy: 0.8129 - output_vowel_accuracy: 0.9324 - output_consonant_accuracy: 0.9349 - val_loss: 0.6299 - val_output_root_loss: 0.4143 - val_output_vowel_loss: 0.0973 - val_output_consonant_loss: 0.1158 - val_output_root_accuracy: 0.8749 - val_output_vowel_accuracy: 0.9709 - val_output_consonant_accuracy: 0.9653
Epoch 2/50
451/451 [==============================] - 35s 77ms/step - loss: 1.0557 - output_root_loss: 0.6510 - output_vowel_loss: 0.2064 - output_consonant_loss: 0.1983 - output_root_accuracy: 0.8130 - output_vowel_accuracy: 0.9350 - output_consonant_accuracy: 0.9355 - val_loss: 0.6289 - val_output_root_loss: 0.4125 - val_output_vowel_loss: 0.0979 - val_output_consonant_loss: 0.1158 - val_output_root_accuracy: 0.8753 - val_output_vowel_accuracy: 0.9711 - val_output_consonant_accuracy: 0.9659
Epoch 3/50
451/

In [ ]:
a=[1,2,3]
b=[4,5,6]
a.extend(b)
print(a)

In [ ]:
#**********************************************************
#standard and ml imports
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from collections import defaultdict
import mlflow
import random
from keras import backend as K
K.tensorflow_backend._get_available_gpus() # ensuring keras uses GPU (rocket emoji)

#**********************************************************
# using custom scripts for creating model and loading data
import sys
sys.path.insert(1, './model/')

# helper function to automate model building
from model_creator import model_create
# helper function to import data
from data_loader import data_loader
# helper function to do image threshold filtering and resizing
from image_processor import image_processor_func
# helper function to start and track mlflow experiments
from experiments import run_experiment_without_callbacks
# function to output final predictions on submission set
from tester import test_func
# function to do data augmentation for multiple output convnets - from this kernel 
# https://www.kaggle.com/kaushal2896/bengali-graphemes-starter-eda-multi-output-cnn
from batch_generator import MultiOutputDataGenerator

In [ ]:
from experiments import run_experiment_with_callbacks

In [ ]:
# defining function to reset graph and set seeds - from Aurelien Gueron's book HandsOn ML
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [ ]:
reset_graph()

In [ ]:
# loading the pre-defined experiment csv
experiment_matrix = pd.read_csv('./experiment_matrix.csv',index_col='Experiment',)
experiment_matrix.head()

In [ ]:
filenames = ['train','test','class_map','class_map_corrected','train_multi_diacritics','sample_submission']
df_dict = defaultdict()

for file in filenames:
    df_dict[file]=pd.read_csv('./data/{}.csv'.format(file))

In [ ]:
# define batch size and number of epochs for training
BATCH_SIZE = 100
EPOCHS = 50

In [ ]:
import mlflow.keras
title = '5_test_with_LRreduction_Patience_10_ES_10'
reset_graph()
model, callbacks_list = model_create(input_shape=(68,118,1),conv_kernel_size=5,
                             pool_size=2,
                            dropout_rate1=0.25,
                             dropout_rate2=0.25,
                            n_conv_layers=4,
                             conv_nfilters=50,early_stopping=True,learningrate_reduction=True)

run_experiment_with_callbacks(title,model,callbacks_list,batch_size=BATCH_SIZE,epochs=EPOCHS,df_dict=df_dict,data_augmentation=True)